In [1]:
import requests
import bs4
import pandas as pd

In [2]:
def make_fs_dataframe(firm_code):
    fs_url='https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=103&stkGb=701&gicode='+firm_code
    fs_page=requests.get(fs_url)
    fs_tables=pd.read_html(fs_page.text)
    
    temp_df=fs_tables[0]
    temp_df=temp_df.set_index(temp_df.columns[0])
    temp_df=temp_df[temp_df.columns[:4]]
    temp_df=temp_df.loc[['매출액','영업이익','당기순이익']]
    
    temp_df2=fs_tables[2]
    temp_df2=temp_df2.set_index(temp_df2.columns[0])
    temp_df2=temp_df2.loc[['자산','부채','자본']]
    
    temp_df3=fs_tables[4]
    temp_df3=temp_df3.set_index(temp_df3.columns[0])
    temp_df3=temp_df3.loc[['영업활동으로인한현금흐름']]
    
    fs_df=pd.concat([temp_df,temp_df2,temp_df3])
    
    return fs_df

In [3]:
def change_df(firm_code,dataframe):
    for num,col in enumerate(fs_df.columns):
        temp_df=pd.DataFrame({code:dataframe[col]})
        temp_df=temp_df.T
        temp_df.columns=[[col]*len(dataframe),temp_df.columns]
        if num==0:
            total_df=temp_df
        else:
            total_df=pd.merge(total_df,temp_df,how='outer',left_index=True,right_index=True)
    return total_df

In [4]:
def make_fr_dataframe(firm_code):
    fr_url = 'https://comp.fnguide.com/SVO2/asp/SVD_FinanceRatio.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=104&stkGb=701&gicode=' + firm_code
    fr_page = requests.get(fr_url)
    fr_tables = pd.read_html(fr_page.text)
    
    temp_df = fr_tables[0]
    
    temp_df = temp_df.set_index(temp_df.columns[0])
    temp_df = temp_df.loc[['유동비율계산에 참여한 계정 펼치기',
                           '부채비율계산에 참여한 계정 펼치기',
                           '영업이익률계산에 참여한 계정 펼치기',
                           'ROA계산에 참여한 계정 펼치기',
                           'ROIC계산에 참여한 계정 펼치기']]
    temp_df.index = ['유동비율', '부채비율', '영업이익률', 'ROA', 'ROIC']
    return temp_df


In [5]:
def make_invest_dataframe(firm_code):
    invest_url = 'https://comp.fnguide.com/SVO2/asp/SVD_Invest.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=105&stkGb=701&gicode=' + firm_code
    invest_page = requests.get(invest_url)
    invest_tables = pd.read_html(invest_page.text)
    temp_df = invest_tables[1]
    
    temp_df = temp_df.set_index(temp_df.columns[0])
    temp_df = temp_df.loc[['PER계산에 참여한 계정 펼치기','PCR계산에 참여한 계정 펼치기','PSR계산에 참여한 계정 펼치기','PBR계산에 참여한 계정 펼치기','총현금흐름']]
    temp_df.index = ['PER', 'PCR', 'PSR', 'PBR', '총현금흐름']
    return temp_df

In [6]:
path=r'C:\Users\lge\Desktop\파이썬연습\data.xls'
code_data=pd.read_excel(path)
code_data=code_data[['종목코드','기업명']]

In [7]:
def make_code(x):
    x=str(x)
    return 'A'+'0'*(6-len(x))+x

code_data['종목코드']=code_data['종목코드'].apply(make_code)

In [8]:
import time

for num,code in enumerate(code_data['종목코드']):
    try:
        print(num,code)
        time.sleep(1)
        try:
            fs_df=make_fs_dataframe(code)
        except requests.exceptions.Timeout:
            time.sleep(60)
            fs_df=make_fs_dataframe(code)
        fs_df_changed=change_df(code,fs_df)
        if num == 0:
            total_fs=fs_df_changed
        else:
            total_fs=pd.concat([total_fs,fs_df_changed])
    except ValueError:
        continue
    except KeyError:
        continue

0 A060310
1 A095570
2 A006840
3 A054620
4 A265520
5 A211270
6 A027410
7 A282330
8 A138930
9 A001460
10 A001040
11 A079160
12 A035760
13 A000120
14 A011150
15 A097950
16 A051500
17 A058820
18 A023460
19 A065770
20 A083660
21 A000590
22 A012030
23 A367340
24 A016610
25 A005830
26 A000990
27 A139130
28 A068790
29 A004840
30 A241520
31 A155660
32 A069730
33 A017940
34 A245620
35 A037370
36 A079190
37 A365550
38 A007700
39 A130500
40 A114090
41 A900290
42 A078930
43 A083450
44 A006360
45 A001250
46 A007070
47 A028150
48 A045890
49 A078150
50 A012630
51 A039570
52 A089470
53 A294870
54 A011200
55 A036640
56 A082740
57 A335870
58 A351340
59 A351320
60 A095340
61 A099520
62 A175330
63 A950170
64 A234080
65 A067290
66 A001060
67 A096760
68 A035900
69 A024840
70 A105560
71 A024120
72 A021320
73 A036670
74 A009440
75 A112190
76 A119650
77 A044180
78 A092220
79 A151860
80 A016380
81 A046440
82 A035600
83 A001390
84 A060720
85 A001940
86 A122450
87 A052900
88 A058400
89 A114450
90 A025000
91 A09223

692 A012690
693 A005360
694 A080420
695 A080160
696 A204210
697 A100030
698 A087260
699 A101330
700 A012860
701 A250060
702 A033200
703 A009680
704 A118990
705 A006920
706 A009580
707 A001810
708 A009200
709 A322970
710 A033920
711 A008420
712 A279600
713 A028040
714 A095500
715 A025560
716 A218150
717 A007120
718 A006800
719 A353490
720 A328380
721 A333430
722 A357250
723 A100790
724 A085620
725 A208890
726 A049950
727 A213090
728 A207760
729 A225850
730 A002840
731 A268280
732 A107590
733 A134380
734 A003650
735 A059090
736 A214610
737 A201490
738 A950190
739 A214180
740 A155900
741 A206640
742 A018700
743 A035620
744 A064520
745 A053030
746 A301300
747 A064550
748 A208710
749 A142760
750 A086820
751 A038460
752 A281310
753 A266470
754 A216400
755 A086040
756 A199290
757 A032980
758 A222160
759 A043150
760 A323990
761 A003610
762 A267790
763 A001340
764 A046310
765 A035150
766 A036620
767 A066410
768 A002410
769 A206400
770 A019010
771 A177350
772 A299910
773 A096300
774 A007210
775 

1346 A140610
1347 A101400
1348 A036570
1349 A217820
1350 A138250
1351 A031860
1352 A238170
1353 A224760
1354 A310840
1355 A319400
1356 A353190
1357 A359090
1358 A365590
1359 A181710
1360 A265740
1361 A354200
1362 A093510
1363 A208860
1364 A183490
1365 A085310
1366 A182400
1367 A009810
1368 A069410
1369 A227950
1370 A198080
1371 A004250
1372 A048830
1373 A096870
1374 A276240
1375 A900140
1376 A061970
1377 A138690
1378 A290650
1379 A066970
1380 A156100
1381 A010120
1382 A311060
1383 A073110
1384 A083310
1385 A297890
1386 A037950
1387 A170920
1388 A058630
1389 A058970
1390 A019590
1391 A169330
1392 A097520
1393 A179290
1394 A225860
1395 A009780
1396 A123040
1397 A323230
1398 A032790
1399 A033160
1400 A347890
1401 A033310
1402 A259630
1403 A115960
1404 A060850
1405 A014440
1406 A007530
1407 A143540
1408 A111770
1409 A009970
1410 A003520
1411 A000670
1412 A036560
1413 A006740
1414 A012280
1415 A265560
1416 A012160
1417 A036000
1418 A250930
1419 A053280
1420 A015360
1421 A122640
1422 A179720

1977 A005740
1978 A215570
1979 A043590
1980 A114120
1981 A110790
1982 A900250
1983 A083790
1984 A045520
1985 A214150
1986 A352770
1987 A237880
1988 A139670
1989 A020120
1990 A311270
1991 A039490
1992 A012170
1993 A054780
1994 A219130
1995 A180060
1996 A065130
1997 A014580
1998 A015890
1999 A006890
2000 A023160
2001 A003240
2002 A011280
2003 A053620
2004 A004100
2005 A116100
2006 A009410
2007 A044490
2008 A124560
2009 A001420
2010 A007980
2011 A191420
2012 A073640
2013 A182690
2014 A066700
2015 A095610
2016 A131970
2017 A055490
2018 A308700
2019 A089030
2020 A258050
2021 A054450
2022 A091440
2023 A078000
2024 A200230
2025 A214420
2026 A215480
2027 A051360
2028 A045340
2029 A108230
2030 A079970
2031 A199800
2032 A105550
2033 A026150
2034 A322180
2035 A117730
2036 A033830
2037 A084870
2038 A057680
2039 A064760
2040 A246710
2041 A340570
2042 A277880
2043 A131290
2044 A019180
2045 A356860
2046 A062860
2047 A363280
2048 A321550
2049 A091810
2050 A004870
2051 A191600
2052 A104480
2053 A309900

In [9]:
total_fs.to_excel(r'C:\Users\lge\Desktop\파이썬연습\재무제표데이터.xlsx')

In [12]:
firmcode_list = ['A005930', 'A005380', 'A035420', 'A003550', 'A034730']
for num, code in enumerate(firmcode_list):
    fs_df = make_fs_dataframe(code)
    fs_df_changed = change_df(code, fs_df)
    if num == 0 :
        total_fs = fs_df_changed
    else:
        total_fs = pd.concat([total_fs, fs_df_changed])
print(total_fs)



firmcode_list = ['A005930', 'A005380', 'A035420', 'A003550', 'A034730']
for num, code in enumerate(firmcode_list):
    fr_df = make_fr_dataframe(code)
    fr_df_changed = change_df(code, fr_df)
    if num == 0 :
        total_fr = fr_df_changed
    else:
        total_fr = pd.concat([total_fr, fr_df_changed])
print(total_fr)



firmcode_list = ['A005930', 'A005380', 'A035420', 'A003550', 'A034730']
for num, code in enumerate(firmcode_list):
    invest_df = make_invest_dataframe(code)
    invest_df_changed = change_df(code, invest_df)
    if num == 0 :
        total_invest = invest_df_changed
    else:
        total_invest = pd.concat([total_invest, invest_df_changed])
print(total_invest)



            2017/12                                                       \
IFRS(연결)        매출액      영업이익     당기순이익         자산         부채         자본   
A005930   2395754.0  536450.0  421867.0  3017521.0   872607.0  2144914.0   
A005380    963761.0   45747.0   45464.0  1781995.0  1034421.0   747574.0   
A035420     46785.0   11792.0    7701.0    80193.0    27141.0    53052.0   
A003550    118411.0   21858.0   24356.0   216477.0    54682.0   161795.0   
A034730    906128.0   57450.0   50665.0  1097835.0   640270.0   457564.0   

                         2018/12                      ...    2019/12  \
IFRS(연결) 영업활동으로인한현금흐름        매출액      영업이익     당기순이익  ...         부채   
A005930      621620.0  2437714.0  588867.0  443449.0  ...   896841.0   
A005380       39224.0   968126.0   24222.0   16450.0  ...  1181465.0   
A035420        9400.0    55869.0    9425.0    6279.0  ...    57956.0   
A003550       10584.0    77434.0   18213.0   18829.0  ...    34136.0   
A034730       69301.0  1001616.0   

In [13]:
for num, code in enumerate(code_data['종목코드']):
    try:
        print(num, code)
        time.sleep(1)
        try:
            fr_df = make_fr_dataframe(code)
        except requests.exceptions.Timeout:
            time.sleep(60)
            fr_df = make_fr_dataframe(code)
        fr_df_changed = change_df(code, fr_df)
        if num == 0 :
            total_fr = fr_df_changed
        else:
            total_fr = pd.concat([total_fr, fr_df_changed])
    except ValueError:
        continue
    except KeyError:
        continue

0 A060310
1 A095570
2 A006840
3 A054620
4 A265520
5 A211270
6 A027410
7 A282330
8 A138930
9 A001460
10 A001040
11 A079160
12 A035760
13 A000120
14 A011150
15 A097950
16 A051500
17 A058820
18 A023460
19 A065770
20 A083660
21 A000590
22 A012030
23 A367340
24 A016610
25 A005830
26 A000990
27 A139130
28 A068790
29 A004840
30 A241520
31 A155660
32 A069730
33 A017940
34 A245620
35 A037370
36 A079190
37 A365550
38 A007700
39 A130500
40 A114090
41 A900290
42 A078930
43 A083450
44 A006360
45 A001250
46 A007070
47 A028150
48 A045890
49 A078150
50 A012630
51 A039570
52 A089470
53 A294870
54 A011200
55 A036640
56 A082740
57 A335870
58 A351340
59 A351320
60 A095340
61 A099520
62 A175330
63 A950170
64 A234080
65 A067290
66 A001060
67 A096760
68 A035900
69 A024840
70 A105560
71 A024120
72 A021320
73 A036670
74 A009440
75 A112190
76 A119650
77 A044180
78 A092220
79 A151860
80 A016380
81 A046440
82 A035600
83 A001390
84 A060720
85 A001940
86 A122450
87 A052900
88 A058400
89 A114450
90 A025000
91 A09223

692 A012690
693 A005360
694 A080420
695 A080160
696 A204210
697 A100030
698 A087260
699 A101330
700 A012860
701 A250060
702 A033200
703 A009680
704 A118990
705 A006920
706 A009580
707 A001810
708 A009200
709 A322970
710 A033920
711 A008420
712 A279600
713 A028040
714 A095500
715 A025560
716 A218150
717 A007120
718 A006800
719 A353490
720 A328380
721 A333430
722 A357250
723 A100790
724 A085620
725 A208890
726 A049950
727 A213090
728 A207760
729 A225850
730 A002840
731 A268280
732 A107590
733 A134380
734 A003650
735 A059090
736 A214610
737 A201490
738 A950190
739 A214180
740 A155900
741 A206640
742 A018700
743 A035620
744 A064520
745 A053030
746 A301300
747 A064550
748 A208710
749 A142760
750 A086820
751 A038460
752 A281310
753 A266470
754 A216400
755 A086040
756 A199290
757 A032980
758 A222160
759 A043150
760 A323990
761 A003610
762 A267790
763 A001340
764 A046310
765 A035150
766 A036620
767 A066410
768 A002410
769 A206400
770 A019010
771 A177350
772 A299910
773 A096300
774 A007210
775 

1346 A140610
1347 A101400
1348 A036570
1349 A217820
1350 A138250
1351 A031860
1352 A238170
1353 A224760
1354 A310840
1355 A319400
1356 A353190
1357 A359090
1358 A365590
1359 A181710
1360 A265740
1361 A354200
1362 A093510
1363 A208860
1364 A183490
1365 A085310
1366 A182400
1367 A009810
1368 A069410
1369 A227950
1370 A198080
1371 A004250
1372 A048830
1373 A096870
1374 A276240
1375 A900140
1376 A061970
1377 A138690
1378 A290650
1379 A066970
1380 A156100
1381 A010120
1382 A311060
1383 A073110
1384 A083310
1385 A297890
1386 A037950
1387 A170920
1388 A058630
1389 A058970
1390 A019590
1391 A169330
1392 A097520
1393 A179290
1394 A225860
1395 A009780
1396 A123040
1397 A323230
1398 A032790
1399 A033160
1400 A347890
1401 A033310
1402 A259630
1403 A115960
1404 A060850
1405 A014440
1406 A007530
1407 A143540
1408 A111770
1409 A009970
1410 A003520
1411 A000670
1412 A036560
1413 A006740
1414 A012280
1415 A265560
1416 A012160
1417 A036000
1418 A250930
1419 A053280
1420 A015360
1421 A122640
1422 A179720

1977 A005740
1978 A215570
1979 A043590
1980 A114120
1981 A110790
1982 A900250
1983 A083790
1984 A045520
1985 A214150
1986 A352770
1987 A237880
1988 A139670
1989 A020120
1990 A311270
1991 A039490
1992 A012170
1993 A054780
1994 A219130
1995 A180060
1996 A065130
1997 A014580
1998 A015890
1999 A006890
2000 A023160
2001 A003240
2002 A011280
2003 A053620
2004 A004100
2005 A116100
2006 A009410
2007 A044490
2008 A124560
2009 A001420
2010 A007980
2011 A191420
2012 A073640
2013 A182690
2014 A066700
2015 A095610
2016 A131970
2017 A055490
2018 A308700
2019 A089030
2020 A258050
2021 A054450
2022 A091440
2023 A078000
2024 A200230
2025 A214420
2026 A215480
2027 A051360
2028 A045340
2029 A108230
2030 A079970
2031 A199800
2032 A105550
2033 A026150
2034 A322180
2035 A117730
2036 A033830
2037 A084870
2038 A057680
2039 A064760
2040 A246710
2041 A340570
2042 A277880
2043 A131290
2044 A019180
2045 A356860
2046 A062860
2047 A363280
2048 A321550
2049 A091810
2050 A004870
2051 A191600
2052 A104480
2053 A309900

In [14]:
total_fr.to_excel(r'C:\Users\lge\Desktop\파이썬연습\재무비율데이터.xlsx')

In [15]:
for num, code in enumerate(code_data['종목코드']):
    try:
        print(num, code)
        time.sleep(1)
        try:
            invest_df = make_invest_dataframe(code)
        except requests.exceptions.Timeout:
            time.sleep(60)
            invest_df = make_invest_dataframe(code)
        invest_df_changed = change_df(code, invest_df)
        if num == 0 :
            total_invest = invest_df_changed
        else:
            total_invest = pd.concat([total_invest, invest_df_changed])
    except ValueError:
        continue
    except KeyError:
        continue
total_invest.to_excel(r'C:\Users\lge\Desktop\파이썬연습\투자지표데이터.xlsx')

0 A060310
1 A095570
2 A006840
3 A054620
4 A265520
5 A211270
6 A027410
7 A282330
8 A138930
9 A001460
10 A001040
11 A079160
12 A035760
13 A000120
14 A011150
15 A097950
16 A051500
17 A058820
18 A023460
19 A065770
20 A083660
21 A000590
22 A012030
23 A367340
24 A016610
25 A005830
26 A000990
27 A139130
28 A068790
29 A004840
30 A241520
31 A155660
32 A069730
33 A017940
34 A245620
35 A037370
36 A079190
37 A365550
38 A007700
39 A130500
40 A114090
41 A900290
42 A078930
43 A083450
44 A006360
45 A001250
46 A007070
47 A028150
48 A045890
49 A078150
50 A012630
51 A039570
52 A089470
53 A294870
54 A011200
55 A036640
56 A082740
57 A335870
58 A351340
59 A351320
60 A095340
61 A099520
62 A175330
63 A950170
64 A234080
65 A067290
66 A001060
67 A096760
68 A035900
69 A024840
70 A105560
71 A024120
72 A021320
73 A036670
74 A009440
75 A112190
76 A119650
77 A044180
78 A092220
79 A151860
80 A016380
81 A046440
82 A035600
83 A001390
84 A060720
85 A001940
86 A122450
87 A052900
88 A058400
89 A114450
90 A025000
91 A09223

692 A012690
693 A005360
694 A080420
695 A080160
696 A204210
697 A100030
698 A087260
699 A101330
700 A012860
701 A250060
702 A033200
703 A009680
704 A118990
705 A006920
706 A009580
707 A001810
708 A009200
709 A322970
710 A033920
711 A008420
712 A279600
713 A028040
714 A095500
715 A025560
716 A218150
717 A007120
718 A006800
719 A353490
720 A328380
721 A333430
722 A357250
723 A100790
724 A085620
725 A208890
726 A049950
727 A213090
728 A207760
729 A225850
730 A002840
731 A268280
732 A107590
733 A134380
734 A003650
735 A059090
736 A214610
737 A201490
738 A950190
739 A214180
740 A155900
741 A206640
742 A018700
743 A035620
744 A064520
745 A053030
746 A301300
747 A064550
748 A208710
749 A142760
750 A086820
751 A038460
752 A281310
753 A266470
754 A216400
755 A086040
756 A199290
757 A032980
758 A222160
759 A043150
760 A323990
761 A003610
762 A267790
763 A001340
764 A046310
765 A035150
766 A036620
767 A066410
768 A002410
769 A206400
770 A019010
771 A177350
772 A299910
773 A096300
774 A007210
775 

1346 A140610
1347 A101400
1348 A036570
1349 A217820
1350 A138250
1351 A031860
1352 A238170
1353 A224760
1354 A310840
1355 A319400
1356 A353190
1357 A359090
1358 A365590
1359 A181710
1360 A265740
1361 A354200
1362 A093510
1363 A208860
1364 A183490
1365 A085310
1366 A182400
1367 A009810
1368 A069410
1369 A227950
1370 A198080
1371 A004250
1372 A048830
1373 A096870
1374 A276240
1375 A900140
1376 A061970
1377 A138690
1378 A290650
1379 A066970
1380 A156100
1381 A010120
1382 A311060
1383 A073110
1384 A083310
1385 A297890
1386 A037950
1387 A170920
1388 A058630
1389 A058970
1390 A019590
1391 A169330
1392 A097520
1393 A179290
1394 A225860
1395 A009780
1396 A123040
1397 A323230
1398 A032790
1399 A033160
1400 A347890
1401 A033310
1402 A259630
1403 A115960
1404 A060850
1405 A014440
1406 A007530
1407 A143540
1408 A111770
1409 A009970
1410 A003520
1411 A000670
1412 A036560
1413 A006740
1414 A012280
1415 A265560
1416 A012160
1417 A036000
1418 A250930
1419 A053280
1420 A015360
1421 A122640
1422 A179720

1977 A005740
1978 A215570
1979 A043590
1980 A114120
1981 A110790
1982 A900250
1983 A083790
1984 A045520
1985 A214150
1986 A352770
1987 A237880
1988 A139670
1989 A020120
1990 A311270
1991 A039490
1992 A012170
1993 A054780
1994 A219130
1995 A180060
1996 A065130
1997 A014580
1998 A015890
1999 A006890
2000 A023160
2001 A003240
2002 A011280
2003 A053620
2004 A004100
2005 A116100
2006 A009410
2007 A044490
2008 A124560
2009 A001420
2010 A007980
2011 A191420
2012 A073640
2013 A182690
2014 A066700
2015 A095610
2016 A131970
2017 A055490
2018 A308700
2019 A089030
2020 A258050
2021 A054450
2022 A091440
2023 A078000
2024 A200230
2025 A214420
2026 A215480
2027 A051360
2028 A045340
2029 A108230
2030 A079970
2031 A199800
2032 A105550
2033 A026150
2034 A322180
2035 A117730
2036 A033830
2037 A084870
2038 A057680
2039 A064760
2040 A246710
2041 A340570
2042 A277880
2043 A131290
2044 A019180
2045 A356860
2046 A062860
2047 A363280
2048 A321550
2049 A091810
2050 A004870
2051 A191600
2052 A104480
2053 A309900